In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1410503,2021-06-27,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1410504,2021-06-28,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1410505,2021-06-29,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1410506,2021-06-30,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
37275,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
37277,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
37279,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
37281,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
37283,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
864123,2021-06-27,Arkansas,Arkansas,5001,2205,38.00,5000,Arkansas,AR,Arkansas,State,3017804
864125,2021-06-28,Arkansas,Arkansas,5001,2211,38.00,5000,Arkansas,AR,Arkansas,State,3017804
864127,2021-06-29,Arkansas,Arkansas,5001,2216,38.00,5000,Arkansas,AR,Arkansas,State,3017804
864129,2021-06-30,Arkansas,Arkansas,5001,2219,38.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1410503,2021-06-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1410504,2021-06-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1410505,2021-06-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1410506,2021-06-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-07-01') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
527,2021-07-01,Snohomish,Washington,40284,605.00,53061,WA,County,822083,4900.24,73.59
1052,2021-07-01,Cook,Illinois,557124,11048.00,17031,IL,County,5150233,10817.45,214.51
1576,2021-07-01,Orange,California,273636,5125.00,6059,CA,County,3175692,8616.58,161.38
2099,2021-07-01,Maricopa,Arizona,560446,10282.00,4013,AZ,County,4485414,12494.86,229.23
2622,2021-07-01,Los Angeles,California,1251383,24499.00,6037,CA,County,10039107,12465.08,244.04
...,...,...,...,...,...,...,...,...,...,...,...
1409583,2021-07-01,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08
1409845,2021-07-01,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1410076,2021-07-01,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1410303,2021-07-01,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
527,2021-07-01,Snohomish,Washington,40284,605.00,53061,WA,County,822083,4900.24,73.59,73.59,4900.24
1052,2021-07-01,Cook,Illinois,557124,11048.00,17031,IL,County,5150233,10817.45,214.51,214.51,10817.45
1576,2021-07-01,Orange,California,273636,5125.00,6059,CA,County,3175692,8616.58,161.38,161.38,8616.58
2099,2021-07-01,Maricopa,Arizona,560446,10282.00,4013,AZ,County,4485414,12494.86,229.23,229.23,12494.86
2622,2021-07-01,Los Angeles,California,1251383,24499.00,6037,CA,County,10039107,12465.08,244.04,244.04,12465.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1409583,2021-07-01,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08,75.08,2627.63
1409845,2021-07-01,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1410076,2021-07-01,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1410303,2021-07-01,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
903688,2021-07-01,Hale,Alabama,2274,78.00,1065,AL,County,14651,15521.12,532.39,532.39,15521.12,1
983933,2021-07-01,Clarke,Alabama,3533,61.00,1025,AL,County,23622,14956.40,258.23,258.23,14956.40,2
706581,2021-07-01,Lowndes,Alabama,1421,54.00,1085,AL,County,9726,14610.32,555.21,555.21,14610.32,3
626315,2021-07-01,Etowah,Alabama,14241,366.00,1055,AL,County,102268,13925.18,357.88,357.88,13925.18,4
525336,2021-07-01,Franklin,Alabama,4326,81.00,1059,AL,County,31362,13793.76,258.27,258.27,13793.76,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862751,2021-07-01,Sublette,Wyoming,797,7.00,56035,WY,County,9831,8107.01,71.20,71.20,8107.01,19
862290,2021-07-01,Converse,Wyoming,1037,18.00,56009,WY,County,13822,7502.53,130.23,130.23,7502.53,20
1072855,2021-07-01,Lincoln,Wyoming,1431,14.00,56023,WY,County,19830,7216.34,70.60,70.60,7216.34,21
1194711,2021-07-01,Niobrara,Wyoming,163,2.00,56027,WY,County,2356,6918.51,84.89,84.89,6918.51,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
706581,2021-07-01,Lowndes,Alabama,1421,54.00,1085,AL,County,9726,14610.32,555.21,555.21,14610.32,3,1
903688,2021-07-01,Hale,Alabama,2274,78.00,1065,AL,County,14651,15521.12,532.39,532.39,15521.12,1,2
624459,2021-07-01,Clay,Alabama,1603,59.00,1027,AL,County,13235,12111.82,445.79,445.79,12111.82,20,3
299948,2021-07-01,Walker,Alabama,7371,282.00,1127,AL,County,63521,11604.04,443.95,443.95,11604.04,29,4
625851,2021-07-01,Dallas,Alabama,3622,159.00,1047,AL,County,37196,9737.61,427.47,427.47,9737.61,55,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072855,2021-07-01,Lincoln,Wyoming,1431,14.00,56023,WY,County,19830,7216.34,70.60,70.60,7216.34,21,19
760289,2021-07-01,Hot Springs,Wyoming,373,3.00,56017,WY,County,4413,8452.30,67.98,67.98,8452.30,17,20
1046010,2021-07-01,Uinta,Wyoming,2320,13.00,56041,WY,County,20226,11470.38,64.27,64.27,11470.38,5,21
298538,2021-07-01,Teton,Wyoming,3801,11.00,56039,WY,County,23464,16199.28,46.88,46.88,16199.28,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,73.59,4900.24,21,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,73.59,4900.24,21,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,73.59,4900.24,21,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,73.59,4900.24,21,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,73.59,4900.24,21,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407213,2021-06-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1407214,2021-06-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1407215,2021-06-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1407216,2021-06-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
899940,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15521.12,2,1,1.00
899941,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15521.12,2,1,0.00
899942,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15521.12,2,1,0.00
899943,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15521.12,2,1,0.00
899944,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,15521.12,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090899,2021-06-27,Crook,Wyoming,458,12.00,56011,WY,County,7584,6039.03,158.23,171.41,6052.22,6,23,0.00
1090900,2021-06-28,Crook,Wyoming,458,12.00,56011,WY,County,7584,6039.03,158.23,171.41,6052.22,6,23,0.00
1090901,2021-06-29,Crook,Wyoming,459,13.00,56011,WY,County,7584,6052.22,171.41,171.41,6052.22,6,23,1.00
1090902,2021-06-30,Crook,Wyoming,459,13.00,56011,WY,County,7584,6052.22,171.41,171.41,6052.22,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
703751,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,1.00,0.00
703752,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,0.00,0.00
703753,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,0.00,0.00
703754,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,0.00,0.00
703755,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701431,2021-06-27,Albany,Wyoming,4615,17.00,56001,WY,County,38880,11869.86,43.72,43.72,11947.02,23,3,0.00,0.00
701432,2021-06-28,Albany,Wyoming,4626,17.00,56001,WY,County,38880,11898.15,43.72,43.72,11947.02,23,3,11.00,0.00
701433,2021-06-29,Albany,Wyoming,4629,17.00,56001,WY,County,38880,11905.86,43.72,43.72,11947.02,23,3,3.00,0.00
701434,2021-06-30,Albany,Wyoming,4637,17.00,56001,WY,County,38880,11926.44,43.72,43.72,11947.02,23,3,8.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-07-01') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
358196,2021-07-01,Imperial,California,28981,741.00,6025,CA,County,181215,15992.61,408.91,408.91,15992.61,1,2,0.00,0.00
2622,2021-07-01,Los Angeles,California,1251383,24499.00,6037,CA,County,10039107,12465.08,244.04,244.04,12465.08,2,5,475.00,1.00
178114,2021-07-01,San Bernardino,California,299883,4975.00,6071,CA,County,2180085,13755.56,228.20,228.20,13755.56,3,4,120.00,35.00
628123,2021-07-01,Inyo,California,1434,38.00,6027,CA,County,18039,7949.44,210.65,210.65,7949.44,4,25,0.00,0.00
79529,2021-07-01,Stanislaus,California,63745,1077.00,6099,CA,County,550660,11576.11,195.58,195.58,11576.11,5,9,24.00,2.00
63245,2021-07-01,San Joaquin,California,74900,1444.00,6077,CA,County,762148,9827.49,189.46,189.46,9827.49,6,17,0.00,0.00
36792,2021-07-01,Riverside,California,301774,4637.00,6065,CA,County,2470546,12214.87,187.69,187.69,12214.87,7,7,68.00,1.00
98646,2021-07-01,Tulare,California,49896,853.00,6107,CA,County,466195,10702.82,182.97,182.97,10702.82,8,10,0.00,1.00
569085,2021-07-01,Merced,California,32380,482.00,6047,CA,County,277680,11660.90,173.58,173.58,11660.90,9,8,11.00,1.00
35828,2021-07-01,Fresno,California,102490,1727.00,6019,CA,County,999101,10258.22,172.86,172.86,10258.22,10,14,26.00,2.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1333753,2021-07-01,Lassen,California,5821,26.00,6035,CA,County,30573,19039.68,85.04,85.04,19039.68,35,1,0.00,0.00
358196,2021-07-01,Imperial,California,28981,741.00,6025,CA,County,181215,15992.61,408.91,408.91,15992.61,1,2,0.00,0.00
762080,2021-07-01,Kings,California,23107,247.00,6031,CA,County,152940,15108.54,161.50,161.50,15108.54,11,3,11.00,0.00
178114,2021-07-01,San Bernardino,California,299883,4975.00,6071,CA,County,2180085,13755.56,228.20,228.20,13755.56,3,4,120.00,35.00
2622,2021-07-01,Los Angeles,California,1251383,24499.00,6037,CA,County,10039107,12465.08,244.04,244.04,12465.08,2,5,475.00,1.00
224037,2021-07-01,Kern,California,111060,1407.00,6029,CA,County,900202,12337.23,156.30,156.30,12337.23,13,6,0.00,0.00
36792,2021-07-01,Riverside,California,301774,4637.00,6065,CA,County,2470546,12214.87,187.69,187.69,12214.87,7,7,68.00,1.00
569085,2021-07-01,Merced,California,32380,482.00,6047,CA,County,277680,11660.90,173.58,173.58,11660.90,9,8,11.00,1.00
79529,2021-07-01,Stanislaus,California,63745,1077.00,6099,CA,County,550660,11576.11,195.58,195.58,11576.11,5,9,24.00,2.00
98646,2021-07-01,Tulare,California,49896,853.00,6107,CA,County,466195,10702.82,182.97,182.97,10702.82,8,10,0.00,1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
358196,2021-07-01,Imperial,California,28981,741.00,6025,CA,County,181215,15992.61,408.91,408.91,15992.61,1,2,0.00,0.00
2622,2021-07-01,Los Angeles,California,1251383,24499.00,6037,CA,County,10039107,12465.08,244.04,244.04,12465.08,2,5,475.00,1.00
178114,2021-07-01,San Bernardino,California,299883,4975.00,6071,CA,County,2180085,13755.56,228.20,228.20,13755.56,3,4,120.00,35.00
628123,2021-07-01,Inyo,California,1434,38.00,6027,CA,County,18039,7949.44,210.65,210.65,7949.44,4,25,0.00,0.00
79529,2021-07-01,Stanislaus,California,63745,1077.00,6099,CA,County,550660,11576.11,195.58,195.58,11576.11,5,9,24.00,2.00
63245,2021-07-01,San Joaquin,California,74900,1444.00,6077,CA,County,762148,9827.49,189.46,189.46,9827.49,6,17,0.00,0.00
36792,2021-07-01,Riverside,California,301774,4637.00,6065,CA,County,2470546,12214.87,187.69,187.69,12214.87,7,7,68.00,1.00
98646,2021-07-01,Tulare,California,49896,853.00,6107,CA,County,466195,10702.82,182.97,182.97,10702.82,8,10,0.00,1.00
569085,2021-07-01,Merced,California,32380,482.00,6047,CA,County,277680,11660.90,173.58,173.58,11660.90,9,8,11.00,1.00
35828,2021-07-01,Fresno,California,102490,1727.00,6019,CA,County,999101,10258.22,172.86,172.86,10258.22,10,14,26.00,2.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
4702,06/27/21,Lassen,5809,25.00,19000.43,81.77,35,1,0.00,0.00
4703,06/28/21,Lassen,5809,26.00,19000.43,85.04,35,1,0.00,1.00
4704,06/29/21,Lassen,5819,26.00,19033.13,85.04,35,1,10.00,0.00
4705,06/30/21,Lassen,5821,26.00,19039.68,85.04,35,1,2.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
4788,06/27/21,Fresno,103091,1727.00,10318.38,172.86,10,14,18.00,0.00
4789,06/28/21,Fresno,103110,1727.00,10320.28,172.86,10,14,19.00,0.00
4790,06/29/21,Fresno,103121,1727.00,10321.38,172.86,10,14,11.00,0.00
4791,06/30/21,Fresno,102464,1725.00,10255.62,172.66,10,14,-657.00,-2.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)